In [608]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# **データの読み込み**

In [908]:
train = pd.read_csv('//Users/naoya5614/Desktop/AI Quest2021 アセスメント/train_cleaning.csv')
test = pd.read_csv('/Users/naoya5614/Desktop/AI Quest2021 アセスメント/test_cleaning.csv')
sample = pd.read_csv('/Users/naoya5614/Desktop/AI Quest2021 アセスメント/sample_submit.csv', header=None)

In [909]:
pd.set_option('display.max_columns', 150)

In [910]:
train.head()

,Unnamed: 0,bed_type,city,cleaning_fee,review_scores_rating,room_type,y,amenities_TV,amenities_Internet,amenities_parking,amenities_24-hour,amenities_Kitchen,amenities_Pets,amenities_Safety card,amenities_Air conditioning,amenities_Heating,amenities_Family,amenities_Self,amenities_{},name_Penthouse,name_Guest,name_Cozy,name_Modern,name_Classic,name_Luxury,name_Artist,name_Private,name_Ocean,name_Great,name_garden,name_pool,accommodates_1,accommodates_10,accommodates_11,accommodates_12,accommodates_13,accommodates_14,accommodates_15,accommodates_16,accommodates_2,accommodates_3,accommodates_4,accommodates_5,accommodates_6,accommodates_7,accommodates_8,accommodates_9,bathrooms_0.0,bathrooms_0.5,bathrooms_1.0,bathrooms_1.5,bathrooms_2.0,bathrooms_2.5,bathrooms_3.0,bathrooms_3.5,bathrooms_4.0,bathrooms_4.5,bathrooms_5.0,bathrooms_5.5,bathrooms_6.0,bathrooms_6.5,bathrooms_7.0,bathrooms_7.5,bathrooms_8.0,bedrooms_0.0,bedrooms_1.0,bedrooms_10.0,bedrooms_2.0,bedrooms_3.0,bedrooms_4.0,bedrooms_5.0,bedrooms_6.0,bedrooms_7.0,bedrooms_8.0,bedrooms_9.0,beds_0.0,beds_1.0,beds_10.0,beds_11.0,beds_12.0,beds_13.0,beds_14.0,beds_15.0,beds_16.0,beds_18.0,beds_2.0,beds_3.0,beds_4.0,beds_5.0,beds_6.0,beds_7.0,beds_8.0,beds_9.0,host_has_profile_pic_f,host_has_profile_pic_t,host_identity_verified_f,host_identity_verified_t,instant_bookable_f,instant_bookable_t,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,property_type_Boutique hotel,property_type_Bungalow,property_type_Cabin,property_type_Camper/RV,property_type_Castle,property_type_Cave,property_type_Chalet,property_type_Condominium,property_type_Dorm,property_type_Guest suite,property_type_Guesthouse,property_type_Hostel,property_type_House,property_type_Hut,property_type_In-law,property_type_Island,property_type_Loft,property_type_Other,property_type_Serviced apartment,property_type_Tent,property_type_Townhouse,property_type_Villa,property_type_Yurt,property_type_other
0,0,0,3,1,-0.312779,0,138.0,1,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0.678682,0,42.0,1,1,1,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2,0,1,1,0.257311,0,65.0,1,1,0,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,2,1,0.554749,0,166.0,1,1,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,1,1,0.678682,1,165.0,1,1,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [911]:
test.head()

,Unnamed: 0,bed_type,city,cleaning_fee,review_scores_rating,room_type,amenities_TV,amenities_Internet,amenities_parking,amenities_24-hour,amenities_Kitchen,amenities_Pets,amenities_Safety card,amenities_Air conditioning,amenities_Heating,amenities_Family,amenities_Self,amenities_{},name_Penthouse,name_Guest,name_Cozy,name_Modern,name_Classic,name_Luxury,name_Artist,name_Private,name_Ocean,name_Great,name_garden,name_pool,accommodates_1,accommodates_10,accommodates_11,accommodates_12,accommodates_13,accommodates_14,accommodates_15,accommodates_16,accommodates_2,accommodates_3,accommodates_4,accommodates_5,accommodates_6,accommodates_7,accommodates_8,accommodates_9,bathrooms_0.0,bathrooms_0.5,bathrooms_1.0,bathrooms_1.5,bathrooms_2.0,bathrooms_2.5,bathrooms_3.0,bathrooms_3.5,bathrooms_4.0,bathrooms_4.5,bathrooms_5.0,bathrooms_5.5,bathrooms_6.0,bathrooms_6.5,bathrooms_7.0,bathrooms_7.5,bathrooms_8.0,bathrooms_nan,bedrooms_0.0,bedrooms_1.0,bedrooms_10.0,bedrooms_2.0,bedrooms_3.0,bedrooms_4.0,bedrooms_5.0,bedrooms_6.0,bedrooms_7.0,bedrooms_8.0,bedrooms_9.0,bedrooms_nan,beds_0.0,beds_1.0,beds_10.0,beds_11.0,beds_12.0,beds_13.0,beds_14.0,beds_15.0,beds_16.0,beds_2.0,beds_3.0,beds_4.0,beds_5.0,beds_6.0,beds_7.0,beds_8.0,beds_9.0,beds_nan,host_has_profile_pic_f,host_has_profile_pic_t,host_identity_verified_f,host_identity_verified_t,instant_bookable_f,instant_bookable_t,property_type_Apartment,property_type_Bed & Breakfast,property_type_Boat,property_type_Boutique hotel,property_type_Bungalow,property_type_Cabin,property_type_Camper/RV,property_type_Castle,property_type_Cave,property_type_Chalet,property_type_Condominium,property_type_Dorm,property_type_Guest suite,property_type_Guesthouse,property_type_Hostel,property_type_House,property_type_Hut,property_type_In-law,property_type_Loft,property_type_Other,property_type_Serviced apartment,property_type_Tent,property_type_Timeshare,property_type_Townhouse,property_type_Villa,property_type_Yurt,property_type_other
0,0,0,5,1,0.430817,1,1,1,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,3,1,0.678682,1,1,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,1,0,-1.824757,0,1,1,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,1,0,-1.824757,1,1,1,0,0,1,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,3,1,0.480390,1,1,1,1,0,1,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


## **データを学習用と検証用に分割**

In [912]:
train = train.drop('Unnamed: 0', axis=1)
train['property_type_Timeshare'] = 0
train['bathrooms_nan'] = 0
train['bedrooms_nan'] = 0
train['beds_nan'] = 0

In [913]:
test['beds_180'] = 0
test['property_type_Island'] = 0

In [914]:
import re
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [915]:
from sklearn.model_selection import train_test_split

# room_type別にデータを分割 train
data_entire = train[train['room_type'] == 0]
data_private = train[train['room_type'] == 1]
data_share = train[train['room_type'] == 2]

X_e = data_entire.drop('y', axis=1)
X_p = data_private.drop('y', axis=1)
X_s = data_share.drop('y', axis=1)

y_e = data_entire['y']
y_p = data_private['y']
y_s = data_share['y']

# データの分割
X_train_e,X_test_e,y_train_e,y_test_e = train_test_split(X_e, y_e, train_size=0.9, random_state = 0)
X_train_p,X_test_p,y_train_p,y_test_p = train_test_split(X_p, y_p, train_size=0.9, random_state = 0)
X_train_s,X_test_s,y_train_s,y_test_s = train_test_split(X_s, y_s, train_size=0.9, random_state = 0)

# **データの分析**

### **分析結果を求める関数**

In [916]:
def result(x_train, x_test, y_train, y_test):
    from sklearn.metrics import mean_squared_error

    #平均二乗誤差
    print('-----RMSE(train)-----')
    print(np.sqrt(mean_squared_error(y_train, model.predict(x_train))))
    print('-----RMSE(test)-----')
    print(np.sqrt(mean_squared_error(y_test, model.predict(x_test))))

## **LGBMRegressor**

In [917]:
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression #重回帰分析
from sklearn.ensemble import RandomForestRegressor #ランダムフォレスト 
from sklearn.metrics import mean_squared_error as MSE

In [918]:
#eデータ
model_e1 = LGBMRegressor(boosting_type='gbdt', num_leaves=30, n_estimators=250, objective='regression')
model_e2 = LinearRegression()
model_e3 = RandomForestRegressor()
model_e4 =GradientBoostingRegressor(min_samples_split=5,
                                  min_samples_leaf=52,
                                  max_depth=6,
                                  max_features='sqrt',
                                  subsample=0.8)

model_e1.fit(X_train_e, y_train_e)
model_e2.fit(X_train_e, y_train_e)
model_e3.fit(X_train_e, y_train_e)
model_e4.fit(X_train_e, y_train_e)

#モデルから予測結果を求める
y_pred_train_e1 = model_e1.predict(X_train_e)
y_pred_test_e1 = model_e1.predict(X_test_e)
y_pred_train_e2 = model_e2.predict(X_train_e)
y_pred_test_e2 = model_e2.predict(X_test_e)
y_pred_train_e3 = model_e3.predict(X_train_e)
y_pred_test_e3 = model_e3.predict(X_test_e)
y_pred_train_e4 = model_e4.predict(X_train_e)
y_pred_test_e4 = model_e4.predict(X_test_e)

print ("LightGBMのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_e, y_pred_train_e1))))
print ("LightGBMのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_e, y_pred_test_e1))))
print ("重回帰分析のRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_e, y_pred_train_e2))))
print ("重回帰分析のRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_e, y_pred_test_e2))))
print ("ランダムフォレストのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_e, y_pred_train_e3))))
print ("ランダムフォレストのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_e, y_pred_test_e3))))
print ("GradientBoostingRegressorのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_e, y_pred_train_e4))))
print ("GradientBoostingRegressorのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_e, y_pred_test_e4))))

LightGBMのRMSE_train: 60.7939
LightGBMのRMSE_test: 63.4937
重回帰分析のRMSE_train: 72.9279
重回帰分析のRMSE_test: 66.5611
ランダムフォレストのRMSE_train: 35.4236
ランダムフォレストのRMSE_test: 66.9935
GradientBoostingRegressorのRMSE_train: 71.3685
GradientBoostingRegressorのRMSE_test: 64.6319


In [919]:
stack_pred_train_e = np.column_stack((y_pred_train_e1,y_pred_train_e3,y_pred_train_e4))
stack_pred_test_e = np.column_stack((y_pred_test_e1,y_pred_test_e3,y_pred_test_e4))

meta_model_e1 = LinearRegression()
meta_model_e2 = LinearRegression()

meta_model_e1.fit(stack_pred_train_e, y_train_e)
meta_model_e2.fit(stack_pred_test_e, y_test_e)

stack_train_pred_e = np.column_stack((y_pred_train_e1,y_pred_train_e3,y_pred_train_e4))
stack_test_pred_e = np.column_stack((y_pred_test_e1,y_pred_test_e3,y_pred_test_e4))
meta_train_pred_e = meta_model_e1.predict(stack_train_pred_e)
meta_test_pred_e = meta_model_e2.predict(stack_test_pred_e)
print ("メタモデルのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_e, meta_train_pred_e))))
print ("メタモデルのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_e, meta_test_pred_e))))

メタモデルのRMSE_train: 30.7677
メタモデルのRMSE_test: 62.6544


In [920]:
#pデータ
model_p1 = LGBMRegressor(boosting_type='gbdt', num_leaves=30, n_estimators=250, objective='regression')
model_p2 = LinearRegression()
model_p3 = RandomForestRegressor()
model_p4 =GradientBoostingRegressor(min_samples_split=5,
                                  min_samples_leaf=52,
                                  max_depth=6,
                                  max_features='sqrt',
                                  subsample=0.8)

model_p1.fit(X_train_p, y_train_p)
model_p2.fit(X_train_p, y_train_p)
model_p3.fit(X_train_p, y_train_p)
model_p4.fit(X_train_p, y_train_p)

#モデルから予測結果を求める
y_pred_train_p1 = model_p1.predict(X_train_p)
y_pred_test_p1 = model_p1.predict(X_test_p)
y_pred_train_p2 = model_p2.predict(X_train_p)
y_pred_test_p2 = model_p2.predict(X_test_p)
y_pred_train_p3 = model_p3.predict(X_train_p)
y_pred_test_p3 = model_p3.predict(X_test_p)
y_pred_train_p4 = model_p4.predict(X_train_p)
y_pred_test_p4 = model_p4.predict(X_test_p)

print ("LightGBMのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_p, y_pred_train_p1))))
print ("LightGBMのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_p, y_pred_test_p1))))
print ("重回帰分析のRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_p, y_pred_train_p2))))
print ("重回帰分析のRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_p, y_pred_test_p2))))
print ("ランダムフォレストのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_p, y_pred_train_p3))))
print ("ランダムフォレストのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_p, y_pred_test_p3))))
print ("GradientBoostingRegressorのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_p, y_pred_train_p4))))
print ("GradientBoostingRegressorのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_p, y_pred_test_p4))))

LightGBMのRMSE_train: 117.3921
LightGBMのRMSE_test: 150.9546
重回帰分析のRMSE_train: 147.3603
重回帰分析のRMSE_test: 160.1337
ランダムフォレストのRMSE_train: 60.5683
ランダムフォレストのRMSE_test: 157.5756
GradientBoostingRegressorのRMSE_train: 143.4705
GradientBoostingRegressorのRMSE_test: 160.7158


In [921]:
stack_pred_train_p = np.column_stack((y_pred_train_p1,y_pred_train_p3,y_pred_train_p4))
stack_pred_test_p = np.column_stack((y_pred_test_p1,y_pred_test_p3,y_pred_test_p4))

meta_model_p1 = LinearRegression()
meta_model_p2 = LinearRegression()

meta_model_p1.fit(stack_pred_train_p, y_train_p)
meta_model_p2.fit(stack_pred_test_p, y_test_p)

stack_train_pred_p = np.column_stack((y_pred_train_p1, y_pred_train_p3,y_pred_train_p4))
stack_test_pred_p = np.column_stack((y_pred_test_p1,y_pred_test_p3,y_pred_test_p4))
meta_train_pred_p = meta_model_p1.predict(stack_train_pred_p)
meta_test_pred_p = meta_model_p2.predict(stack_test_pred_p)
print ("メタモデルのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_p, meta_train_pred_p))))
print ("メタモデルのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_p, meta_test_pred_p))))

メタモデルのRMSE_train: 47.1896
メタモデルのRMSE_test: 150.2490


In [922]:
#sデータ
model_s1 = LGBMRegressor(boosting_type='gbdt', num_leaves=30, n_estimators=250, objective='regression')
model_s2 = LinearRegression()
model_s3 = RandomForestRegressor()
model_s4 =GradientBoostingRegressor(min_samples_split=5,
                                  min_samples_leaf=52,
                                  max_depth=6,
                                  max_features='sqrt',
                                  subsample=0.8)

model_s1.fit(X_train_s, y_train_s)
model_s2.fit(X_train_s, y_train_s)
model_s3.fit(X_train_s, y_train_s)
model_s4.fit(X_train_s, y_train_s)

#モデルから予測結果を求める
y_pred_train_s1 = model_s1.predict(X_train_s)
y_pred_test_s1 = model_s1.predict(X_test_s)
y_pred_train_s2 = model_s2.predict(X_train_s)
y_pred_test_s2 = model_s2.predict(X_test_s)
y_pred_train_s3 = model_s3.predict(X_train_s)
y_pred_test_s3 = model_s3.predict(X_test_s)
y_pred_train_s4 = model_s4.predict(X_train_s)
y_pred_test_s4 = model_s4.predict(X_test_s)

print ("LightGBMのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_s, y_pred_train_s1))))
print ("LightGBMのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_s, y_pred_test_s1))))
print ("重回帰分析のRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_s, y_pred_train_s2))))
print ("重回帰分析のRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_s, y_pred_test_s2))))
print ("ランダムフォレストのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_s, y_pred_train_s3))))
print ("ランダムフォレストのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_s, y_pred_test_s3))))
print ("GradientBoostingRegressorのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_s, y_pred_train_s4))))
print ("GradientBoostingRegressorのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_s, y_pred_test_s4))))

LightGBMのRMSE_train: 48.1131
LightGBMのRMSE_test: 94.0001
重回帰分析のRMSE_train: 90.7165
重回帰分析のRMSE_test: 218923827128.3528
ランダムフォレストのRMSE_train: 40.5607
ランダムフォレストのRMSE_test: 98.9953
GradientBoostingRegressorのRMSE_train: 89.3692
GradientBoostingRegressorのRMSE_test: 81.5586


In [923]:
stack_pred_train_s = np.column_stack((y_pred_train_s1,y_pred_train_s3,y_pred_train_s4))
stack_pred_test_s = np.column_stack((y_pred_test_s1,y_pred_test_s3,y_pred_test_s4))

meta_model_s1 = LinearRegression()
meta_model_s2 = LinearRegression()

meta_model_s1.fit(stack_pred_train_s, y_train_s)
meta_model_s2.fit(stack_pred_test_s, y_test_s)

stack_train_pred_s = np.column_stack((y_pred_train_s1,y_pred_train_s3,y_pred_train_s4))
stack_test_pred_s = np.column_stack((y_pred_test_s1,y_pred_test_s3,y_pred_test_s4))
meta_train_pred_s = meta_model_s1.predict(stack_train_pred_s)
meta_test_pred_s = meta_model_s2.predict(stack_test_pred_s)
print ("メタモデルのRMSE_train: {:.4f}".format(np.sqrt(MSE(y_train_s, meta_train_pred_s))))
print ("メタモデルのRMSE_test: {:.4f}".format(np.sqrt(MSE(y_test_s, meta_test_pred_s))))

メタモデルのRMSE_train: 28.4880
メタモデルのRMSE_test: 81.3883


In [924]:
# room_type別にデータを分割 test
sub_entire = test[test['room_type'] == 0]
sub_private = test[test['room_type'] == 1]
sub_share = test[test['room_type'] == 2]

In [925]:
sub_entire.head()

,Unnamed0,bed_type,city,cleaning_fee,review_scores_rating,room_type,amenities_TV,amenities_Internet,amenities_parking,amenities_24hour,amenities_Kitchen,amenities_Pets,amenities_Safetycard,amenities_Airconditioning,amenities_Heating,amenities_Family,amenities_Self,amenities_,name_Penthouse,name_Guest,name_Cozy,name_Modern,name_Classic,name_Luxury,name_Artist,name_Private,name_Ocean,name_Great,name_garden,name_pool,accommodates_1,accommodates_10,accommodates_11,accommodates_12,accommodates_13,accommodates_14,accommodates_15,accommodates_16,accommodates_2,accommodates_3,accommodates_4,accommodates_5,accommodates_6,accommodates_7,accommodates_8,accommodates_9,bathrooms_00,bathrooms_05,bathrooms_10,bathrooms_15,bathrooms_20,bathrooms_25,bathrooms_30,bathrooms_35,bathrooms_40,bathrooms_45,bathrooms_50,bathrooms_55,bathrooms_60,bathrooms_65,bathrooms_70,bathrooms_75,bathrooms_80,bathrooms_nan,bedrooms_00,bedrooms_10,bedrooms_100,bedrooms_20,bedrooms_30,bedrooms_40,bedrooms_50,bedrooms_60,bedrooms_70,bedrooms_80,bedrooms_90,bedrooms_nan,beds_00,beds_10,beds_100,beds_110,beds_120,beds_130,beds_140,beds_150,beds_160,beds_20,beds_30,beds_40,beds_50,beds_60,beds_70,beds_80,beds_90,beds_nan,host_has_profile_pic_f,host_has_profile_pic_t,host_identity_verified_f,host_identity_verified_t,instant_bookable_f,instant_bookable_t,property_type_Apartment,property_type_BedBreakfast,property_type_Boat,property_type_Boutiquehotel,property_type_Bungalow,property_type_Cabin,property_type_CamperRV,property_type_Castle,property_type_Cave,property_type_Chalet,property_type_Condominium,property_type_Dorm,property_type_Guestsuite,property_type_Guesthouse,property_type_Hostel,property_type_House,property_type_Hut,property_type_Inlaw,property_type_Loft,property_type_Other,property_type_Servicedapartment,property_type_Tent,property_type_Timeshare,property_type_Townhouse,property_type_Villa,property_type_Yurt,property_type_other,beds_180,property_type_Island
2,2,0,1,0,-1.824757,0,1,1,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,7,0,3,1,0.331670,0,0,1,0,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,8,0,2,0,0.629109,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
11,11,0,1,0,0.678682,0,1,1,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,13,0,0,1,0.554749,0,1,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [926]:
sub_entire_id = sub_entire.iloc[:, 0].reset_index(drop=True)
sub_private_id = sub_private.iloc[:, 0].reset_index(drop=True)
sub_share_id = sub_share.iloc[:, 0].reset_index(drop=True)

In [927]:
sub_entire = sub_entire.drop('Unnamed0', axis=1)
sub_private = sub_private.drop('Unnamed0', axis=1)
sub_share = sub_share.drop('Unnamed0', axis=1)

In [928]:
sub_pred_e1 = model_e1.predict(sub_entire)
sub_pred_e3 = model_e3.predict(sub_entire)
sub_pred_e4 = model_e4.predict(sub_entire)

stack_pred_sub_e = np.column_stack((sub_pred_e1,sub_pred_e3,sub_pred_e4))
sub_pred_e = pd.DataFrame(meta_model_e1.predict(stack_pred_sub_e))

In [929]:
sub_pred_p1 = model_p1.predict(sub_private)
sub_pred_p3 = model_p3.predict(sub_private)
sub_pred_p4 = model_p4.predict(sub_private)

stack_pred_sub_p = np.column_stack((sub_pred_p1,sub_pred_p3,sub_pred_p4))
sub_pred_p = pd.DataFrame(meta_model_p1.predict(stack_pred_sub_p))

In [930]:
sub_pred_s1 = model_s1.predict(sub_share)
sub_pred_s3 = model_s3.predict(sub_share)
sub_pred_s4 = model_s4.predict(sub_share)

stack_pred_sub_s = np.column_stack((sub_pred_s1,sub_pred_s3,sub_pred_s4))
sub_pred_s = pd.DataFrame(meta_model_s1.predict(stack_pred_sub_s))

In [931]:
sub_pred_e

,0
0,559.897476
1,198.628049
2,71.995078
3,67.318733
4,105.436941
...,...
7610,98.302181
7611,86.907395
7612,42.605030
7613,138.157304


In [932]:
sub_entire_id 

0           2
1           7
2           8
3          11
4          13
        ...  
7610    18516
7611    18517
7612    18518
7613    18519
7614    18520
Name: Unnamed0, Length: 7615, dtype: int64

In [933]:
sub_pred_e = pd.concat([sub_entire_id, sub_pred_e], axis=1)
sub_pred_p = pd.concat([sub_private_id, sub_pred_p], axis=1)
sub_pred_s = pd.concat([sub_share_id, sub_pred_s], axis=1)

In [934]:
sub_pred_e

,Unnamed0,0
0,2,559.897476
1,7,198.628049
2,8,71.995078
3,11,67.318733
4,13,105.436941
...,...,...
7610,18516,98.302181
7611,18517,86.907395
7612,18518,42.605030
7613,18519,138.157304


In [935]:
sub_pred_p

,Unnamed0,0
0,0,321.493894
1,1,176.658973
2,3,158.718446
3,4,576.836501
4,5,127.776059
...,...,...
10366,18522,234.146788
10367,18523,80.452278
10368,18524,135.657655
10369,18525,369.214184


In [936]:
sub_pred_s

,Unnamed0,0
0,16,80.390110
1,105,80.181487
2,128,180.909296
3,131,254.773054
4,158,63.074923
...,...,...
537,18338,49.046412
538,18434,137.070937
539,18462,76.090978
540,18486,91.929434


In [937]:
sub_pred = pd.DataFrame(np.concatenate([sub_pred_e, sub_pred_p, sub_pred_s]))

In [938]:
sub_pred.sort_values(0, inplace=True)

In [939]:
sub_pred = sub_pred.reset_index(drop=True)

In [940]:
sub_pred

,0,1
0,0.0,321.493894
1,1.0,176.658973
2,2.0,559.897476
3,3.0,158.718446
4,4.0,576.836501
...,...,...
18523,18523.0,80.452278
18524,18524.0,135.657655
18525,18525.0,369.214184
18526,18526.0,134.941866


In [941]:
sub_pred = sub_pred.drop(0, axis=1)

In [942]:
sample[1] = sub_pred 
print(sample.head())
sample.to_csv('/Users/naoya5614/Desktop/AI Quest2021 アセスメント/submit_2.csv', index=False, header=False)

   0           1
0  0  321.493894
1  1  176.658973
2  2  559.897476
3  3  158.718446
4  4  576.836501
